# Observational Studies

To draw causal conclusions on the impact of our features on movie success, we have to perform the standard operations used in observational studies, such as propensity matching and regression.

## Packages

In [1]:
# Global packages
import pandas as pd
import numpy as np
# Statistical package
import statsmodels.api as sm
# Matching package
from psmpy import PsmPy
from psmpy.functions import cohenD
from psmpy.plotting import *
# Custom helpers
import feature_and_regression as feat_and_reg
%load_ext autoreload
%autoreload 2

## Load Data

In [12]:
# First gather the initial regression dataframe.
# The dictionnary passed as parameter contains the word to add as 
# covariates in the model. For further information pleaser refer to
# the description of the function.
raw_regression_df = feat_and_reg.get_raw_regression_df({"zombie":True,"alien":False})

In [24]:
# Format the regression dataframe and keep only the given decades.
# For further tuning, check the function description to see the list of available
# parameters.
processed_df, target, binary_target, num_votes = feat_and_reg.format_regression_df(
                                                            raw_regression_df,[2000])
processed_df.head()

,plot_has_zombie,plot_has_alien,North America and Australia,Western Europe,Asia,Africa and Middle-East,Eastern Europe and Russia,Central and South America,gender_ratio,has_famous_actor,...,thriller,horror,animation,children,adult,fantasy,genre,title_length,combinned_movie_num,combinned_movie_success
movie_id,,,,,,,,,,,,,,,,,,,,,
975900,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,-0.294118,1.0,...,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.321755,14.0,1
21926710,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.321755,1.0,0
156558,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.043478,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.365557,4.0,0
25960460,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.166667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.321755,2.0,1
5894429,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,-0.272727,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.321755,10.0,1


## Propensity Score

In [15]:
psm = PsmPy(processed_df.reset_index(), treatment='plot_has_zombie', indx='movie_id', exclude = [])

In [32]:
# Compute propensity scores
# If this cell fails to run, try with balance=False. But otherwise let balance=True.
psm.logistic_ps(balance = False)

In [33]:
# Creates matching
psm.knn_matched(matcher='propensity_score', replacement=False, caliper=None)
matched_ids = set(psm.matched_ids["movie_id"]).union(psm.matched_ids["matched_ID"])

In [34]:
# Regression
features = feat_and_reg.forward_selection(processed_df, target)
model = sm.OLS(target, sm.add_constant(processed_df["plot_has_zombie"])).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:         average_rating   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     2.063
Date:                Sat, 17 Dec 2022   Prob (F-statistic):              0.151
Time:                        18:54:41   Log-Likelihood:                -5309.5
No. Observations:                3405   AIC:                         1.062e+04
Df Residuals:                    3403   BIC:                         1.064e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               6.1478      0.020    3

## Observationnal study pipeline

In [31]:
def binarize_treatment(processed_dataframe: pd.DataFrame,
                       treatment: str, threshold: float, method="greater"):
    """
    
    :param processed_dataframe: Pandas DataFrame containing the data for regression.
    :param treatment: Covariate used as treament (should be binary).
    :param threshold: Threshold value for the binarization.
    :param method: Method for binarization with respect to the threshold.
    """
    if method == "greater":
        f = lambda l: l > threshold
    elif method == "smaller":
        f = lambda l: l < threshold
    elif method == "equal":
        f = lambda l: l == threshold
    elif method == "unequal":
        f = lambda l: l != threshold
    else:
        ValueError("Please provide a method in [greater,smaller,equal,unequal].")
    processed_dataframe[treatment] = processed_dataframe[treatment].apply(f)
    processed_dataframe[treatment] = processed_dataframe[treatment].replace({True: 1, False: 0})
    
def temporal_obs_study(raw_regression_df: pd.DataFrame, treatment: str,
                       decades: list, binarize=False, balance=True,
                       threshold=0, method="greater",alpha=0.05) -> tuple:
    """
    Perform observational study accross the decades provided.
    
    The output will be a dictionnary containing the model for both regular
    regression on the formated dataframe (i.e. with all samples in the decade)
    and another model with only the matched pairs. The second element in the tuple
    is a Pandas DataFrame which gives the result of the signficance and coefficient
    values for regular regression and with matched pairs. It allows to detect
    difference that arise from bias and confounds in the data.
    
    :param raw_regression_df: Pandas DataFrame with raw data for regression.
    :param treatment: Covariate used as treament (should be binary).
    :param decades: List of decades for which the function will perform the analysis.
    :param binarize: Indicator if a binirazation of the treatment is necessary.
    :param balance: Indicator if the logistic regression model should be balanced.
    :param threshold: Threshold value for the binarization.
    :param method: Method for binarization with respect to the threshold.
    :param alpha: Significance level, default 0.05.
    
    :return: Tuple with dictionnary with the model for each decade, and a summary dataframe.  
    
    """
    decades_models = dict()
    decade_significance = dict()
    for decade in decades:
        processed_df, target, binary_target, num_votes = feat_and_reg.format_regression_df(
                                                                raw_regression_df,[decade])
        # If necessary, binarize treatment
        if binarize:
            binarize_treatment(processed_df, treatment,
                               threshold, method=method)
        if (processed_df[treatment].sum() == 0 or 
            processed_df[treatment].sum() == len(processed_df)):
            print("Sample have all undergoe same treatment in data.")
            decade_significance[decade] = (False, None)
            continue
        # Compute propensity scores
        psm = PsmPy(processed_df.reset_index(), treatment=treatment, indx='movie_id')
        psm.logistic_ps(balance=balance)
        # Creates matching
        psm.knn_matched(matcher='propensity_score', replacement=False, caliper=None)
        matched_ids = set(psm.matched_ids["movie_id"]).union(psm.matched_ids["matched_ID"])
        matched_df = processed_df[processed_df.index.isin(matched_ids)]
        matched_targets = target[processed_df.index.isin(matched_ids)]
        # Regressions
        features_regular = feat_and_reg.forward_selection(processed_df, target)
        model_regular = sm.OLS(target, sm.add_constant(processed_df[features_regular])).fit()
        features_matched = feat_and_reg.forward_selection(matched_df, matched_targets)
        model_matched = sm.OLS(matched_targets, sm.add_constant(matched_df[features_matched])).fit()
        # Update results
        decades_models[decade] = (model_matched,model_regular)
        reg_sig, reg_coeff, matched_sig, matched_coeff = False, None, False, None
        if treatment in model_regular.pvalues:
            reg_sig, reg_coeff = (model_regular.pvalues[treatment] < alpha,
                                          model_regular.params[treatment])
        if treatment in model_matched.pvalues:
            reg_sig, reg_coeff = (model_matched.pvalues[treatment] < alpha,
                                          model_matched.params[treatment])
        decade_significance[decade] = (reg_sig, reg_coeff, matched_sig, matched_coeff)
    decade_results_df = pd.DataFrame(decade_significance.values(),index=decade_significance.keys(),
                                    columns=["regular_treatment_significant","regular_coeff",
                                             "matched_treatment_significant","matched_coeff"]).sort_index()
    return decades_models, decade_results_df

In [36]:
model_list, decade_df = temporal_obs_study(
    raw_regression_df,"gender_ratio",[1970,1980,1990,2000,2010],binarize=True,
    balance=True,threshold=0,method="smaller",alpha=0.05)

C:\Users\jerem\anaconda3\envs\ada\lib\site-packages\psmpy\psmpy.py:349: UserWarning: Some values do not have a match
  warnings.warn('Some values do not have a match')


In [37]:
decade_df

,regular_treatment_significant,regular_coeff,matched_treatment_significant,matched_coeff
1970,False,None,False,None
1980,False,None,False,None
1990,False,None,False,None
2000,False,None,False,None
2010,False,None,False,None


---